In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers/S20_tokenizer"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers/S20_tokenizer


In [3]:
from tokenizer.basic_bpe import BasicTokenizer
from tokenizer.regex_bpe import RegexTokenizer

In [5]:
file_name = "tiny_shakespeare.txt"
file_path = os.path.join(os.getcwd(), "data", file_name)

with open(file_path, "r") as f:
    text = f.read()

In [6]:
print(f"The length of the text is {len(text)}")

The length of the text is 1115394


creating the model file for basic tokenizer

In [10]:
model_path = os.path.join(os.getcwd(), "tokenizer_model")
os.makedirs(model_path, exist_ok=True)

In [9]:
basic_tokenizer = BasicTokenizer()
basic_tokenizer.train(text, vocab_size=512, verbose=True)

merge 1/256: (101, 32) -> 256 (b'e ') had 27643 occurrences
merge 2/256: (116, 104) -> 257 (b'th') had 22739 occurrences
merge 3/256: (116, 32) -> 258 (b't ') had 16508 occurrences
merge 4/256: (115, 32) -> 259 (b's ') had 15364 occurrences
merge 5/256: (100, 32) -> 260 (b'd ') had 14165 occurrences
merge 6/256: (44, 32) -> 261 (b', ') had 14098 occurrences
merge 7/256: (111, 117) -> 262 (b'ou') had 12730 occurrences
merge 8/256: (101, 114) -> 263 (b'er') had 11771 occurrences
merge 9/256: (105, 110) -> 264 (b'in') had 10606 occurrences
merge 10/256: (121, 32) -> 265 (b'y ') had 10283 occurrences
merge 11/256: (97, 110) -> 266 (b'an') had 10197 occurrences
merge 12/256: (58, 10) -> 267 (b':\n') had 8762 occurrences
merge 13/256: (111, 114) -> 268 (b'or') had 8458 occurrences
merge 14/256: (111, 32) -> 269 (b'o ') had 8134 occurrences
merge 15/256: (101, 110) -> 270 (b'en') had 7568 occurrences
merge 16/256: (10, 10) -> 271 (b'\n\n') had 7098 occurrences
merge 17/256: (97, 114) -> 272 (

In [11]:
prefix = os.path.join(model_path, "shakespeare_basic")
basic_tokenizer.save(prefix)

Regex tokenizer

In [12]:
regex_tokenizer = RegexTokenizer()
regex_tokenizer.train(text, vocab_size=512, verbose=True)

merge 1/256: (32, 116) -> 256 (b' t') had 23837 occurrences
merge 2/256: (104, 101) -> 257 (b'he') had 18203 occurrences
merge 3/256: (32, 97) -> 258 (b' a') had 13541 occurrences
merge 4/256: (111, 117) -> 259 (b'ou') had 12730 occurrences
merge 5/256: (32, 115) -> 260 (b' s') had 12287 occurrences
merge 6/256: (32, 109) -> 261 (b' m') had 10786 occurrences
merge 7/256: (105, 110) -> 262 (b'in') had 10606 occurrences
merge 8/256: (32, 119) -> 263 (b' w') had 10546 occurrences
merge 9/256: (114, 101) -> 264 (b're') had 9843 occurrences
merge 10/256: (104, 97) -> 265 (b'ha') had 9673 occurrences
merge 11/256: (58, 10) -> 266 (b':\n') had 8762 occurrences
merge 12/256: (110, 100) -> 267 (b'nd') had 8730 occurrences
merge 13/256: (256, 257) -> 268 (b' the') had 8684 occurrences
merge 14/256: (32, 98) -> 269 (b' b') had 8463 occurrences
merge 15/256: (105, 115) -> 270 (b'is') had 7526 occurrences
merge 16/256: (111, 114) -> 271 (b'or') had 7297 occurrences
merge 17/256: (10, 10) -> 272 (b'

In [13]:
prefix = os.path.join(model_path, "shakespeare_regex")
regex_tokenizer.save(prefix)

loading the tokenizer files and using it

In [6]:
model_path = os.path.join(os.getcwd(), "tokenizer_model")
model_path = os.path.join(model_path, "shakespeare_basic.model")

In [7]:
basic_tokenizer = BasicTokenizer()
basic_tokenizer.load(model_path)

In [15]:
ids  = basic_tokenizer.encode("Hello World")

In [19]:
reverse = basic_tokenizer.decode(ids)

In [20]:
mapping = [(str(i),basic_tokenizer.decode([i])) for i in ids]

In [22]:
def test_tokenizer(text):
    ids = basic_tokenizer.encode(text)
    decoded = basic_tokenizer.decode(ids)
    mapping = [(str(i), basic_tokenizer.decode([i])) for i in ids]

    return ids, decoded, mapping

In [23]:
test_tokenizer("Hello World")

([72, 389, 269, 87, 268, 108, 100],
 'Hello World',
 [('72', 'H'),
  ('389', 'ell'),
  ('269', 'o '),
  ('87', 'W'),
  ('268', 'or'),
  ('108', 'l'),
  ('100', 'd')])

In [12]:
import gradio as gr

In [24]:
with gr.Blocks() as demo:
    gr.HTML("<h1 align = 'center'> Token Generation for Hindi Dataset </h1>")

    with gr.Row():
      with gr.Column():
        inputs = [gr.TextArea(label = "Enter initial text to generate tokens in Hindi", lines = 10)]
        generate_btn = gr.Button(value = 'Generate Text')
      with gr.Column():
        enc = gr.Textbox(label = "Encoded Tokens")
        txt = gr.Textbox(label = "Decoded Text from tokens")
        map = gr.Textbox(label = "Mapping of the tokens and respective texts")
        outputs  = [
            enc,
            txt,
            map
            ]
    
    generate_btn.click(fn = test_tokenizer, inputs= inputs, outputs = outputs)


In [27]:
demo.launch(share=True) 


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on public URL: https://9ce8a9238acdc2d242.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
